In [6]:
import shapefile as sp
from dbfread import DBF

Un sigle qui signifiait « Ilots Regroupés pour l'Information Statistique » et qui faisait référence à la taille visée de 2 000 habitants par maille élémentaire.

Depuis, l'IRIS (appellation qui se substitue désormais à IRIS2000) constitue la brique de base en matière de diffusion de données infra-communales. Il doit respecter des critères géographiques et démographiques et avoir des contours identifiables sans ambigüité et stables dans le temps.

Les communes d'au moins 10 000 habitants et une forte proportion des communes de 5 000 à 10 000 habitants sont découpées en IRIS. Ce découpage constitue une partition de leur territoire. La France compte environ 16 100 IRIS dont 650 dans les DOM.

On distingue trois types d'IRIS :

Les IRIS d'habitat : leur population se situe en général entre 1 800 et 5 000 habitants. Ils sont homogènes quant au type d'habitat et leurs limites s'appuient sur les grandes coupures du tissu urbain (voies principales, voies ferrées, cours d'eau...).
Les IRIS d'activité : ils regroupent plus de 1 000 salariés et comptent au moins deux fois plus d'emplois salariés que de population résidente.
Les IRIS divers : il s'agit de grandes zones spécifiques peu habitées et ayant une superficie importante (parcs de loisirs, zones portuaires, forêts...).



In [7]:
class Iris():
    
    def __init__(self):
        self.get_files()
        self.get_records()
        self.get_iris_id()
    
    def get_files(self):
        #client = storage.Client().bucket('trampact_storage')

        #path_dbf = "gs://{}/{}/{}".format(bucket_name, path,'IRIS_GEO_2018_FRTOT.dbf')
        #path_shp = 'gs://{}/{}/{}'.format(bucket_name, path,'IRIS_GEO_2018_FRTOT.shp')
        
        path_dbf ='../raw_data/iris_geo/IRIS_GEO_2018_FRTOT.dbf'
        path_shp ='../raw_data/iris_geo/IRIS_GEO_2018_FRTOT.shp'

        self.dbf=DBF(path_dbf)
        self.shp = sp.Reader(path_shp)


    def get_records(self):
        records = []
        for record in self.dbf:
            records.append(dict(record))
        self.records = records

    def get_iris_id(self):
        iris_id = []
        for i in range(len(self.records)):
            r = self.records[i]['CODE_IRIS']
            iris_id.append(r)
        self.iris_id = iris_id
        
    def get_iris_type(self, iris_id):
        iris_type = {}
        for i in range(len(self.records)):
            iris_type[self.iris_id[i]]= self.records[i]['TYP_IRIS']
        self.iris_type = iris_type
        return iris_type[str(iris_id)]

    def get_box_coord(self, iris_id):
        shapes = self.shp.shapes()
        box_coord = {}
        for i in range(len(shapes)):
            box_coord[self.iris_id[i]] = shapes[i].bbox
        return box_coord[str(iris_id)]

    def get_poly_coord(self, iris_id):
        shapes = self.shp.shapes()
        poly_coord = {}
        for i in range(len(shapes)):
            poly_coord[self.iris_id[i]] = shapes[i].__geo_interface__['coordinates'][0]
        return poly_coord[str(iris_id)]   


In [8]:
iris = Iris()

In [31]:
iris.get_iris_type('061230107')

'H'

In [29]:
iris.get_box_coord('060880801')

[7.288164256383002, 43.70616376439394, 7.323476112735955, 43.7325002458164]

In [10]:
path_dbf ='../raw_data/iris_geo/IRIS_GEO_2018_FRTOT.dbf'
path_shp ='../raw_data/iris_geo/IRIS_GEO_2018_FRTOT.shp'

dbf=DBF(path_dbf)
shp = sp.Reader(path_shp)

records = []
for record in DBF(path_dbf):
    records.append(dict(record))
    

In [11]:
records[0]

{'INSEE_COM': '72191',
 'NOM_COM': 'Mayet',
 'IRIS': '0000',
 'CODE_IRIS': '721910000',
 'NOM_IRIS': 'Mayet',
 'TYP_IRIS': 'Z'}

In [12]:
def get_iris_list():
    iris_id = []
    for i in range(len(records)):
        r = records[i]['CODE_IRIS']
        iris_id.append(r)
    return iris_id

In [22]:
iris_list =get_iris_list()
iris_list.index('2A0990000')
    

45

In [28]:
iris_list.index('060880801')

48412

In [87]:
iris.get_box_coord('010270000')

[5.074076567058182, 45.797890833135114, 5.141096735822965, 45.8552780627737]

In [1]:
len('010270000')

9

In [27]:
len('060880801')

9

In [4]:
len('60880801')

8

In [24]:
'61620000' in iris_list

False

In [6]:
!pip install shapely

In [7]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

True


In [26]:
import pandas as pd
df=pd.read_csv("gs://trampact_storage/data/BPE_T1_v1.csv")
df.head(1)

,Unnamed: 0,Unnamed: 0.1,iris,iris_with_t1,AN,longitude,latitude,"('LAB', 'is_sport')","('LAB', 'is_leasure')","('LAB', 'is_medical')",...,"('LAB', 'is_high_educ')","('LAB', 'is_schools')","('LAB', 'is_transport')",Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise,LAB,CLASSE,distance tram t1,proche tram t1
0,0,4384,60881201,1,2018,7.256436,43.72625,0,0,0,...,0,0,0,0.0,1.0,0.0,"Banque, Caisse d'épargne",Services a,86.91,oui


In [11]:
point = Point(45.829124, 5.102229)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

False


In [31]:
bucket_name = 'trampact_storage'
path = 'data/iris_geo'
    
    
class Iris():
    
    
    def __init__(self):
        self.get_files(bucket_name, path)
        self.get_records()
        self.get_iris_id()
    
    def get_files(self,bucket_name, path):
        #client = storage.Client().bucket('trampact_storage')

        #path_dbf = "gs://{}/{}/{}".format(bucket_name, path,'IRIS_GEO_2018_FRTOT.dbf')
        #path_shp = 'gs://{}/{}/{}'.format(bucket_name, path,'IRIS_GEO_2018_FRTOT.shp')
        path_dbf = 'gs://{}/{}/{}'.format(bucket_name, path,'IRIS_GEO_2018_FRTOT.dbf')
        path_shp = 'gs://{}/{}/{}'.format(bucket_name, path,'IRIS_GEO_2018_FRTOT.shp')
        #path_dbf ='../raw_data/iris_geo/IRIS_GEO_2018_FRTOT.dbf'
        #path_shp ='../raw_data/iris_geo/IRIS_GEO_2018_FRTOT.shp'

        self.dbf=DBF(path_dbf)
        self.shp = sp.Reader(path_shp)
        
    def get_records(self):
        records = []
        for record in self.dbf:
            records.append(dict(record))
        self.records = records

    def get_iris_id(self):
        iris_id = []
        for i in range(len(self.records)):
            r = self.records[i]['CODE_IRIS']
            iris_id.append(r)
        self.iris_id = iris_id

    def get_box_coord(self, iris_id):
        shapes = self.shp.shapes()
        box_coord = {}
        for i in range(len(shapes)):
            box_coord[self.iris_id[i]] = shapes[i].bbox
        return box_coord[str(iris_id)]

    def get_poly_coord(self, iris_id):
        shapes = self.shp.shapes()
        poly_coord = {}
        for i in range(len(shapes)):
            poly_coord[self.iris_id[i]] = shapes[i].__geo_interface__['coordinates'][0]
        return poly_coord[str(iris_id)]